In [1]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

import glob as glob
import os as os
import subprocess as sp
import numpy as np
import shutil
import json
import base64
import pandas as pd

import logging
from importlib import reload
reload(logging)
logging.basicConfig(
    format='%(asctime)s %(message)s',
    datefmt='%Y-%m-%d %I:%M:%S %p',
    level=logging.DEBUG)
logging.info('Started logging...')

import parmed as pmd
import pytraj as pt

from setup.anchor_atoms import grep_anchor_atoms
from setup.restraints import setup_static_restraints, setup_conformation_restraints, setup_guest_restraints, setup_guest_wall_restraints
from paprika.analysis import fe_calc


2018-08-21 04:21:56 PM Started logging...
2018-08-21 04:22:02 PM OpenMM support: Yes


In [2]:
def json_numpy_obj_hook(dct):
    """Decodes a previously encoded numpy ndarray with proper shape and dtype.

    :param dct: (dict) json encoded ndarray
    :return: (ndarray) if input was an encoded ndarray
    """
    if isinstance(dct, dict) and '__ndarray__' in dct:
        data = base64.b64decode(dct['__ndarray__'])
        return np.frombuffer(data, dct['dtype']).reshape(dct['shape'])
        # return dct['__ndarray__']
    return dct


In [3]:
results_files = sorted(glob.glob("results/enthalpy/*.json"))
results_files = [file for file in results_files if "release" not in file]

In [4]:
len(results_files)

86

In [11]:
chemical_types = {
    'aliphatic_ammoniums': ['a-bam', 'a-nmb', 'a-mba', 'a-pam', 'a-ham', 'a-nmh', 'a-mha', 'a-hpa', 'a-mhp', 'a-oam', 'b-ham', 'b-mha', 'b-oam'],
    'cyclic_alcohols': ['a-cbu', 'a-cpe', 'a-chp', 'a-coc', 'b-cbu', 'b-cpe', 'b-mch', 'b-m4c', 'b-m4t', 'b-chp', 'b-coc'],
    'aliphatic_carboxylates': ['a-but', 'a-pnt', 'a-hex', 'a-hx2', 'a-hx3', 'a-hep', 'a-hp6', 'a-oct', 'b-pnt', 'b-hex', 'b-hep', 'b-ben', 
                              'b-pha', 'b-mp3', 'b-mp4', 'b-mo3', 'b-mo4', 'b-pb3', 'b-pb4'],
}

In [12]:
guest_types = {}
for k, v in chemical_types.items():
    for i in v:
        guest_types[i] = k

In [21]:
df = pd.DataFrame()

for file in results_files:
    system = "-".join(os.path.basename(file).split("-")[0:3])
    with open(file, "r") as f:
        results = json.loads(f.read(), object_hook=json_numpy_obj_hook)
        print(results["attach"]["ti-block"]["n_frames"])
    with open(f"results/pre-enthalpy/{system[0]}-release.json", "r") as f:
        release = json.loads(f.read(), object_hook=json_numpy_obj_hook)
        release_fe = release["release"]["ti-block"]["fe"]
        release_sem = release["release"]["ti-block"]["sem"]
    
    smirnoff_fe = -1 * (results["attach"]["ti-block"]["fe"] + results["pull"]["ti-block"]["fe"] - release_fe + results["ref_state_work"])
    smirnoff_sem = np.sqrt(results["attach"]["ti-block"]["sem"]**2 + results["pull"]["ti-block"]["sem"]**2 + release_sem**2)
    
    prefix = os.path.join('systems', system, 'bgbg-tip3p')
    bgbg_tip3p_attach  = np.genfromtxt(os.path.join(prefix, 'ti-a.dat'))
    bgbg_tip3p_pull    = np.genfromtxt(os.path.join(prefix, 'ti-u.dat'))
    bgbg_tip3p_release = np.genfromtxt(os.path.join(prefix, 'ti-r.dat'))
    bgbg_tip3p_analytic = 7.14

    bgbg_tip3p_fe = -1 * (bgbg_tip3p_attach[-1,1] + bgbg_tip3p_pull[-1,1] - bgbg_tip3p_release[-1,1] - bgbg_tip3p_analytic)
    bgbg_tip3p_sem = np.sqrt(bgbg_tip3p_attach[-1,2]**2 + bgbg_tip3p_pull[-1,2]**2 + bgbg_tip3p_release[-1,2]**2)
    
    print(f"{smirnoff_fe:4f} +/- {smirnoff_sem:4f}")
    print(f"{bgbg_tip3p_fe:4f} +/- {bgbg_tip3p_sem:4f}")
    
    guest = "-".join(system.split("-")[0:2])
    
    df = df.append(pd.DataFrame({'system': system,
                                 'smirnoff_fe' : smirnoff_fe,
                                 'smirnoff_sem': smirnoff_sem,
                                 'bgbg_tip3p_fe': bgbg_tip3p_fe,
                                 'bgbg_tip3p_sem': bgbg_tip3p_sem,
                                 'guest_type': guest_types[guest]}, index=[0]), ignore_index=True)

2280000
-3.235478 +/- 0.451365
-0.016103 +/- 0.234765
2280000
-0.543050 +/- 0.380773
-0.616992 +/- 0.270263
2280000
0.203689 +/- 0.298335
-0.335027 +/- 0.312059
2280000
-1.439269 +/- 0.287627
-0.846324 +/- 0.255411
2280000
-0.787973 +/- 0.289186
-0.264900 +/- 0.265886
2276918
-0.978117 +/- 0.235632
-0.579944 +/- 0.300961
2280000
-0.813979 +/- 0.337570
0.263536 +/- 0.269686
2276559
-2.318731 +/- 0.307080
-1.665232 +/- 0.254026
2278084
0.466613 +/- 0.296290
0.342746 +/- 0.307959
2278875
-1.761758 +/- 0.301014
-1.842100 +/- 0.251035
2276552
0.343356 +/- 0.247243
-1.117450 +/- 0.466155
2276558
-1.563468 +/- 0.263083
-0.936360 +/- 0.238919
2278895
-3.478479 +/- 0.525028
-2.258214 +/- 0.244382
2279583
-2.867676 +/- 0.526063
-2.177589 +/- 0.242761
2279339
-3.024338 +/- 0.287611
-2.824313 +/- 0.245619
2277886
-3.776074 +/- 0.252947
-3.806485 +/- 0.237242
2278197
-1.803398 +/- 0.305500
-2.465166 +/- 0.327354
2277138
-2.524681 +/- 0.259653
-2.487976 +/- 0.242336
2276522
-2.586238 +/- 0.302132
-2

In [15]:
df.to_csv("results/enthalpy/results.csv")

In [20]:
results["attach"]["ti-block"]["n_frames"]

2277066